In [4]:
from dataloader import DataLoader

In [5]:
loader = DataLoader()

In [6]:
loader.load_sells_data_from_file('sprzedaz.csv')
df_filtered = loader = loader.prepare_sells_data('0001')

ValueError: Length mismatch: Expected axis has 101 elements, new values have 40 elements